In [4]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from tqdm import tqdm
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 홀 채우기 함수 수정
def mask_filling(mask):
    # 마스크 내부 영역 추출
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_mask = np.zeros_like(mask)
    cv2.drawContours(filled_mask, contours, -1, 255, -1)
    return filled_mask

# CSV 파일 로드
csv_file = pd.read_csv('../submit/deeplabv3plus_bbrn101_224split_manyaug.csv')  # CSV 파일 경로

# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

# 각 사진과 마스크에 대해 예시 출력 및 결과 저장
for i in tqdm(range(60640)):
    # 사진 파일 경로 생성
    
    image_path = f'../data/test_img/{csv_file["img_id"][i]}.png'  # 이미지 파일 경로 생성 (확장자에 따라 수정해야 할 수 있습니다)

    # 사진 파일 확인
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {image_path}")
    except Exception as e:
        print(str(e))
        assert False

    # 마스크 정보 로드 및 디코딩
    mask_data = csv_file['mask_rle'][i]
    mask = rle_decode(mask_data, image.shape[:2])

    # 작은 객체 제거
    min_object_size = 10  # 임의의 작은 객체 크기 기준 설정
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_object_size:
            cv2.drawContours(mask, [contour], 0, 0, -1)

    # 마스크 내부 영역 채우기
    filled_mask = mask_filling(mask)

    # 이미지에 마스크 적용
    #masked_image = cv2.bitwise_and(image, image, mask=filled_mask)

    # 결과 이미지들을 리스트에 추가
    #result_images = [image, mask, filled_mask, masked_image]

    # 결과 이미지 출력
    # fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    # titles = ['Original Image', 'Mask', 'Filled Mask', 'Masked Image']
    # for ax, img, title in zip(axes, result_images, titles):
    #     if len(img.shape) == 3:
    #         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #     ax.imshow(img)
    #     ax.set_title(title)
    #     ax.axis('off')
    # plt.tight_layout()
    # plt.show()

    # 결과를 DataFrame에 저장
    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(filled_mask)]

# 결과 DataFrame을 final.csv 파일로 저장
result_data.to_csv('yjy_post_processing_final.csv', index=False)

100%|██████████| 60640/60640 [06:35<00:00, 153.22it/s]


In [13]:
csv_file = pd.read_csv('./yjy_post_processing_final.csv')  # CSV 파일 경로
csv_file['mask_rle'] = csv_file['mask_rle'].fillna(-1)
#print(type(csv_file['mask_rle'][1]))
csv_file.to_csv('yjy_post_processing_final.csv', index=False)